In [8]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("fpl_data.db")

query = """
SELECT
    players.first_name || ' ' || players.second_name AS name,
    players.now_cost,
    players.selected_by_percent,
    teams.name AS team_name,
    element_types.name as position
FROM players
JOIN teams ON players.team_id = teams.id
JOIN element_types ON players.element_type = element_types.id
WHERE selected_by_percent > 10 and now_cost < 6
ORDER BY selected_by_percent DESC
LIMIT 10
"""

df = pd.read_sql_query(query, conn)
df


,name,now_cost,selected_by_percent,team_name,position
0,Caoimhín Kelleher,4.5,32.2,Brentford,Goalkeepers
1,Aaron Wan-Bissaka,4.5,32.1,West Ham,Defenders
2,Ezri Konsa Ngoyo,4.5,24.0,Aston Villa,Defenders
3,David Raya Martín,5.5,18.6,Arsenal,Goalkeepers
4,Ola Aina,5.0,18.2,Nott'm Forest,Defenders
5,Matz Sels,5.0,17.8,Nott'm Forest,Goalkeepers
6,Norberto Bercique Gomes Betuncal,5.5,15.6,Everton,Forwards
7,Marc Guéhi,4.5,13.7,Crystal Palace,Defenders
8,Robert Lynch Sánchez,5.0,13.5,Chelsea,Goalkeepers
9,Micky van de Ven,4.5,13.4,Spurs,Defenders


In [1]:
%%sql
WITH next_opponents AS (
    SELECT
        f.id,
        f.event,
        CASE
            WHEN f.home_team = t.id THEN f.away_team
            ELSE f.home_team
        END AS opponent_id,
        t.id as team_id
    FROM fixtures f
    JOIN teams t ON t.id = f.home_team OR t.id = f.away_team
    WHERE f.event IN (1, 2, 3)
),
opponents_named AS (
    SELECT
        nf.team_id,
        GROUP_CONCAT(t2.name, ', ') as next_opponents
    FROM next_opponents nf
    JOIN teams t2 ON t2.id = nf.opponent_id
    GROUP BY nf.team_id
)
SELECT
    p.first_name || ' ' || p.second_name AS name,
    p.now_cost,
    p.selected_by_percent,
    t.name AS team_name,
    et.name AS position,
    o.next_opponents
FROM players p
JOIN teams t ON p.team_id = t.id
JOIN element_types et ON p.element_type = et.id
LEFT JOIN opponents_named o ON o.team_id = p.team_id
WHERE p.element_type = 1 AND p.now_cost <= 4.0
ORDER BY p.selected_by_percent DESC
LIMIT 30;

,name,now_cost,selected_by_percent,team_name,position,next_opponents
0,Martin Dúbravka,4,11.3,Newcastle,Goalkeepers,"Aston Villa, Liverpool, Leeds"
1,Karl Darlow,4,8.2,Leeds,Goalkeepers,"Everton, Arsenal, Newcastle"
2,Antonín Kinský,4,6.2,Spurs,Goalkeepers,"Burnley, Man City, Bournemouth"
3,Tom King,4,3.3,Wolves,Goalkeepers,"Man City, Bournemouth, Everton"
4,Hákon Rafn Valdimarsson,4,2.2,Brentford,Goalkeepers,"Nott'm Forest, Aston Villa, Sunderland"
5,Matt Turner,4,1.5,Nott'm Forest,Goalkeepers,"Brentford, Crystal Palace, West Ham"
6,Daniel Bentley,4,1.1,Wolves,Goalkeepers,"Man City, Bournemouth, Everton"
7,Joe Gauci,4,1.0,Aston Villa,Goalkeepers,"Newcastle, Brentford, Crystal Palace"
8,Wes Foderingham,4,1.0,West Ham,Goalkeepers,"Sunderland, Chelsea, Nott'm Forest"
9,Will Dennis,4,0.7,Bournemouth,Goalkeepers,"Liverpool, Wolves, Spurs"
